# Cameron Station Commuter Invoice Creator

The purpose of the code in here is to generate nice and easy invoices for our various commuters throughout a given period of time (usually a calendar month).

Ultimately, this code should record rides by date, sort them into rider-specific data frames, and spit out a nicely formatted string/invoice object (if I can find a Python package for that) that will tell a given rider the dates and times (e.g. Morning or Evening) that they rode with us, plus the total amount they owe us.

**First things first, export the Commuter Google calendar as `calendar_data.ics` so we can ingest it!**

## Imports and Constants

In [ ]:
from icalendar import Calendar, Event
from os import path
import datetime as dt
import pandas as pd

In [ ]:
MORNING_COMMUTE_TIME = dt.time(hour = 8, minute = 0)
EVENING_COMMUTE_TIME = dt.time(hour = 17, minute = 15)

#Rate (in USD) per commuter per ride
GOING_RATE = 3

# NASA garage parking permit cost in USD
PARKING_PERMIT_COST = 50

## Ingesting Google Calendar export

OK, now that we have our calender file, let's continue by defining a function that will allow us to pull in icalendar data objects and spit out useful strings.

In [ ]:
test = "Passenger #1: KC"
passenger = pd.Series(test.split(":")).str.strip()

# Make DataFrame with rows that represent string portions before and after colon
# and columns split on whitespace from pre- and post-colon substrings
separated_text = passenger.str.split(" ", expand=True)
passenger_name = separated_text.loc[1,0]
extra_info = separated_text.loc[1,1:].str.cat(sep = " ")

# If no driver named, assume it's Becky
if separated_text.loc[0,0] == "Passenger":
    driver_name = "Becky"

else:
    driver_name = separated_text.loc[0,0]
    
separated_text

In [ ]:
delta = dt.timedelta(hours = 3)

def record_maker(event_summary, event_datetime):
    """
    Extract info about commuter event and return in a useful format
    
    event_summary: icalendar component summary vText describing the title of the event. Expected
                    to be of the format "Passenger #X: Passenger_Name"
    event_date: icalendar component datetime value. Provides info on both the date of the commute
                and the time (morning vs. evening) that the commute took place.
                
    Returns list of format [Date, Commute Time, Passenger Name]
    """
    
    #[Driver_Name Passenger Passenger_Num, Passenger_Name Extra_Info]
    passenger_info = pd.Series(event_summary.split(":")).str.strip()
    
    # Make DataFrame with rows that represent string portions before and after colon
    # and columns split on whitespace from pre- and post-colon substrings
    separated_text = passenger_info.str.split(" ", expand=True)
    
    passenger_name = separated_text.loc[1,0]
    extra_info = separated_text.loc[1,1:].str.cat(sep = " ")

    # If no driver named, assume it's Becky
    if separated_text.loc[0,0] == "Passenger":
        driver_name = "Becky"

    else:
        driver_name = separated_text.loc[0,0]
    
    
    t = event_datetime
    delta = dt.timedelta(hours = 2)
    
    #Is the time of the commute within +/- delta hours of the usual time?
    if (t + delta).time() >= MORNING_COMMUTE_TIME and (t - delta).time() <= MORNING_COMMUTE_TIME:
        commute_time = "Morning"        
        
    elif (t + delta).time() >= EVENING_COMMUTE_TIME and (t - delta).time() <= EVENING_COMMUTE_TIME:
        commute_time = "Evening"
    else:
        commute_time = "ERROR! Event time doesn't match commute windows!"
        
    #Check to make sure it actually found a name to match to
    if len(passenger_info) > 1 and passenger_info[1].strip() != "":
        output = [event_datetime.date().strftime("%x"), 
                  commute_time, 
                  driver_name.title().strip(),
                  passenger_name.title().strip(),
                 extra_info]
        
        return output
    
           
    else:
        return []

# You must change the start date for invoicing

In [ ]:
#The start and end dates the invoice should cover. The invoice will be INCLUSIVE of these dates
invoice_startdate = dt.date(2019,10,1)



from dateutil.relativedelta import relativedelta
invoice_enddate = invoice_startdate + relativedelta(months = +1, days = -1)

data_dict = {"Date": [],
            "Commute Time": [],
             "Driver Name": [],
            "Passenger Name": [],
            "Extra Info": []}

with open('calendar_data.ics','rb') as f:
    gcal = Calendar.from_ical(f.read())
    for component in gcal.walk():
        temp_data = []
        
        #Check to make sure we skip useless calendar data
        if component.get('summary'):
            summary = component.get('summary')
            dtstart = component.get('dtstart').dt
            
            #Check to make sure we're only looking at the period of time we want to invoice
            if dtstart.date() >= invoice_startdate and dtstart.date() <= invoice_enddate:
                temp_data = record_maker(summary, dtstart)
                
                #Make sure we're only recording records that include actual passengers
                if temp_data:
                    [data_dict[k].append(temp_data[i]) for i,k in enumerate(data_dict.keys())]

## Making a Nice Date- and Time-Resolved Data Frame

In [ ]:
#TODO: add in Driver Name and Extra Info columns
full_record = pd.DataFrame(data_dict)
full_record.sort_values(["Date", "Driver Name"], inplace=True)

# Paul can be...inconsistent, at times.
full_record.replace({'Pg': "Paul"}, inplace=True)
full_record

## Time to Generate That Invoice

Likely the easiest thing to do here is to output an Excel file that only spits out dates of rides and commute segments (e.g. Morning or Evening) for each rider, making a new sheet for each new rider name. Then I can copy and paste those data sets into my Word-based invoice template and VOILA! Done.

In [ ]:
#Write each passenger's data as its own sheet/tab in the output Excel file

# Save in path Commuting/<current_year>/YYYY-MM-DD_CommuterInvoices.xlsx
base_path = "/Users/emigre459/Dropbox/Shared Folder - Becky and Dave/Finances/Commuting"
filepath = path.join(base_path, str(dt.date.today().year), dt.date.today().isoformat() + "_CommuterInvoices.xlsx")

writer = pd.ExcelWriter(filepath)

#Iterate through the unique passenger names in the dataframe
for name in full_record["Passenger Name"].unique():
    temp_df = full_record.groupby("Passenger Name").get_group(name)
    #Remove the Name field, as that will already be in the sheet name
    temp_df.to_excel(writer, sheet_name = name, index = False)

writer.save()

## Taking Care of the Final Bits

At this point, an Excel file should existig in the working directory (same directory as this notebook) that contains a sheet for each unique passenger name from the time period specified. At this stage, all that remains is to do the following:

1. Run the next block of code (which will show the final amount each rider owes in USD) 
3. Copy that Excel document into the Finances/Commuting folder (an alias exists in the working directory for ease of use)
2. Copy the data in each Excel sheet into an invoice Word doc for each passenger who owes money this month, using `CommuterInvoice_Template.docx` as the template for each invoice and using the naming schema `MM-YYYY_CommuterInvoice_PassengerName.docx`. 
    * Don't forget to put in the final amount they owe for that time period/month!


In [ ]:
#How much does each rider owe??
invoice = full_record.groupby(["Driver Name", "Passenger Name"]).count()["Commute Time"] * GOING_RATE

# Becky gets $25 credit for rides with KC due to 
# him owing us half of the monthly parking permit cost

# Did Becky ride with him?
if invoice['Kc'].index.str.contains("Becky").sum() > 0:
    invoice['Kc', 'Becky'] -= int(PARKING_PERMIT_COST / 2)

else:
    invoice['Kc', 'Becky'] = -int(PARKING_PERMIT_COST / 2)
    
    
invoice